<a href="https://colab.research.google.com/github/tomonari-masada/course-nlp2020/blob/master/11_topic_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 標準使用ライブラリー
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc
import os
import shutil
from icecream import ic
from tqdm import tqdm_notebook as tqdm 



# 追記
import json
import datetime
import math

# debug
#%pdb on

import pixiedust #%pixie_debugger

# tfがエラーはかないため
# tfがエラーはかないため
#import tensorflow as tf
#import os
#os.environ["CUDA_VISIBLE_DEVICES"]="5"
#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)


Pixiedust database opened successfully


In [2]:
import pandas as pd
import re
import emoji
from wordcloud import WordCloud, STOPWORDS

df = pd.read_excel("../data/result0802.xlsx")

type(df["description"])
docs = df["description"].to_list()

In [3]:
print(len(docs))

93069


### Neologdを使ってtokenizeする

In [4]:

import subprocess
import MeCab

def make_neologd_tagger():
    cmd='echo `mecab-config --dicdir`"/mecab-ipadic-neologd"'
    path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
                               shell=True).communicate()[0]).decode('utf-8')
    m=MeCab.Tagger("-Ochasen -d "+str(path_neologd))
    return (m)


def neolog_prep_text( text, m):
    return_words = []

    
    splited_text = (re.split('[\t,]', line) for line in m.parse(text).split('\n'))
    for tmp_word in splited_text :
        if (tmp_word[0] in ('EOS', '', 't', 'ー') ):
           continue 
        if not re.match( '名詞' ,tmp_word[3]  ) or tmp_word[0] in emoji.UNICODE_EMOJI["en"]:
            continue
        else:
            return_words.append(tmp_word[0])

    return return_words


* tokenizationの実行

In [5]:
from tqdm import tqdm


m = make_neologd_tagger()

new_docs = list()
for doc in tqdm(docs):
  if str(doc) == "nan":
    continue
  tmp_words =  neolog_prep_text(str(doc), m)
  new_docs.append( tmp_words )
  

100%|██████████| 93069/93069 [00:23<00:00, 4004.35it/s]


* tokenizationの結果を確認

In [6]:
print(new_docs[5])

['ボカロ', '初音ミク', 'アニメ', 'ゲーム', '等', '好き', '20代', '社会人', 'ミク', '廃', 'さん', '達', '中心', 'フォロー', 'ミク', 'キャンバス', 'よろしくお願いします', '相方', '好き', 'カープ', '好き']


* 各文書を長い文字列で表しなおす（CountVectorizerを後で使うため）

In [7]:
corpus = [' '.join(doc) for doc in new_docs]

## 11-02 データ行列の作成
* LDAの場合、単に単語の出現頻度を重みとして各文書をベクトル化する。

### sklearnのCountVectorizerで疎行列化する

* 全文書の半分より多い文書に現れる単語は、高頻度語とみなして削除する。
* 30件未満の文書にしか現れない単語は、低頻度語とみなして削除する。

In [8]:
import os
import urllib.request
def download_stopwords(path):
    url = 'http://svn.sourceforge.jp/svnroot/slothlib/CSharp/Version1/SlothLib/NLP/Filter/StopWord/word/Japanese.txt'
    if os.path.exists(path):
        print('File already exists.')
    else:
        print('Downloading...')
        # Download the file from `url` and save it locally under `file_name`:
        urllib.request.urlretrieve(url, path)

def create_stopwords(file_path):
    stop_words = []
    for w in open(path, "r"):
        w = w.replace('\n','')
        if len(w) > 0:
          stop_words.append(w)
    return stop_words    

path = "stop_words.txt"
download_stopwords(path)
stop_words = create_stopwords(path)

File already exists.


In [9]:
from sklearn.feature_extraction.text import CountVectorizer

MIN_DF = 30

vectorizer = CountVectorizer(max_df=0.5, min_df= MIN_DF, stop_words=stop_words)

X = vectorizer.fit_transform(corpus)

In [10]:
print(len(vectorizer.get_feature_names()))

5050


* 文書数と語彙サイズを変数にセット

In [11]:
n_samples, n_features = X.shape

### TF-IDFで各文書における単語の重みを計算する

In [12]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer()
Xtfidf = tfidf.fit_transform(X)

In [13]:
print(Xtfidf[0])

  (0, 4595)	0.19276647176236258
  (0, 4587)	0.2674347536590711
  (0, 4375)	0.21747308136578852
  (0, 4170)	0.23235458283624436
  (0, 3856)	0.6269365304095426
  (0, 3044)	0.17976188275291347
  (0, 2944)	0.2683528929608284
  (0, 2666)	0.1882646033429383
  (0, 2603)	0.1657289328698403
  (0, 2423)	0.2209442473554379
  (0, 1546)	0.3033883831127719
  (0, 1427)	0.22979355219086722
  (0, 1415)	0.18995772072964873


In [14]:
Xtfidf.shape

(87733, 5050)

### LDAのインポート

In [15]:
from sklearn.decomposition import NMF, LatentDirichletAllocation
from time import time
from matplotlib.backends.backend_pdf import PdfPages
import datetime
from tmtoolkit.topicmod.evaluate import metric_coherence_gensim

### トピックの重要語を取り出す関数の定義

In [16]:
def get_top_words(model, feature_names, n_top_words=30):
  top_features = list()
  weights = list()
  for topic_idx, topic in enumerate(model.components_):
    top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
    top_features.append([feature_names[i] for i in top_features_ind])
    weights.append(topic[top_features_ind])
  return top_features, weights

# LDAでトピック抽出

### LDAによるトピック抽出の実行

In [17]:
def make_word_cloud(n_components, lda):
    # matplotlib and seaborn for plotting
    import matplotlib.pyplot as plt
    %matplotlib inline
    import seaborn as sns
    plt.style.use('dark_background')
    top_words, weights = get_top_words(lda, vectorizer.get_feature_names())
    topic_words = [dict(zip(top_words[i], weights[i])) for i in range(n_components)]
    FONT_PATH = "/usr/share/fonts/opentype/ipaexfont-mincho/ipaexm.ttf"
    cloud = WordCloud(stopwords=STOPWORDS,
                  font_path=FONT_PATH,
                  background_color='white',
                  width=2500,
                  height=1800,
                  max_words=100,
                  colormap='tab10'
                  )

    tate = math.ceil(len(topic_words) / 2)
    fig, axes = plt.subplots(tate, 2, figsize=(32, 50), sharex=True, sharey=True)

    for i, ax in enumerate(axes.flatten()):

        if i >= len(topic_words):
            break

        fig.add_subplot(ax)
        cloud.generate_from_frequencies(topic_words[i], max_font_size=500)
        plt.gca().imshow(cloud)
        plt.gca().set_title('Topic ' + str(i), fontdict=dict(size=16))
        plt.gca().axis('off')

    plt.subplots_adjust(wspace=0, hspace=0)
    plt.axis('off')
    plt.margins(x=0, y=0)
    plt.tight_layout()

    pdf = PdfPages( 
                    now_ +'topic.pdf')

    fignums = plt.get_fignums()
    for fignum in fignums:
        plt.figure(fignum)
        pdf.savefig()

    pdf.close() 


In [18]:
import logging
import pickle
import itertools

def lda_main (now_, batch_size ,n_components, topic_word_prior,doc_topic_prior  ,max_iter=30):

    folder_name = now_

    # フォルダを作成
    os.mkdir("../experiment/0803expt/"+folder_name)
    os.chdir("../experiment/0803expt/"+folder_name)


    
    
    lda = LatentDirichletAllocation(n_components=n_components, 
                                    max_iter=max_iter,
                                    topic_word_prior=topic_word_prior, # トピック数の逆数が目安の0.01,0.02,0.05,0.1などなど試す
                                    doc_topic_prior =  doc_topic_prior, 
                                    learning_method='online',
                                    learning_offset=50,
                                    batch_size= batch_size,# 多くする
                                    learning_decay = 0.7,
                                    mean_change_tol=1e-4,
                                    random_state=1,
                                    evaluate_every=1,
                                    verbose=1)
    print((f"Fitting LDA models with tf features, "
    f"n_samples={n_samples} and n_features={n_features}"))
    t0 = time()
    lda.fit(X)
    print(f"done in {time() - t0:0.3f}s.")
    # パラメータの比較はperplexity
    # ハイパーパラメータ調整を頑張る！（やってられない！といわない！！）
    
    
    coherance = metric_coherence_gensim(measure='c_v', 
#                         top_n=20, # これはデフォルトが20
                        topic_word_distrib=lda.components_, 
                        dtm=Xtfidf,  # tfidfの結果
                        vocab=np.array([x for x in vectorizer.vocabulary_.keys()]), 
                        texts=new_docs)
    
    
    results = {
            "perplexity" : lda.perplexity(X) ,
            "coherance": coherance,
        }

    logger.warning('TIME:{0}'.format(now_) )
    logger.warning('MIN_DF:{0}'.format(MIN_DF) )
    logger.warning('params:batch_size:{0}'.format(batch_size)) 
    logger.warning('params:n_components:{0}'.format(n_components)) 
    logger.warning('params:topic_word_prior:{0}'.format(topic_word_prior)) 
    logger.warning('params:doc_topic_prior:{0}'.format(doc_topic_prior)) 
    logger.warning('params:max_iter:{0}'.format(max_iter)) 
    logger.warning('done n_iter:{0}'.format(lda.n_iter_)) 
    logger.warning('perplexity:{0}'.format(results["perplexity"])) 
    logger.warning('coherance:{0}'.format(results["coherance"]) )
    logger.warning('check all params:{0}'.format(lda.get_params() )) 
    make_word_cloud(n_components, lda)
    # pickle
    file_name = now_ + '_lda.pickle'
    with open(file_name, mode="wb") as f:
        pickle.dump(lda, f)
    
#     breakpoint()
    
    os.chdir("../../")
    return(results)
    

In [ ]:
# logging
logger = logging.getLogger()
fhandler = logging.FileHandler(filename='../experiment/0803expt/mylog.log', mode='a')
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
fhandler.setFormatter(formatter)
logger.addHandler(fhandler)
logger.setLevel(logging.WARNING)
import gc 

for batch_size,n_components, topic_word_prior, doc_topic_prior in itertools.product([1500] ,[6, 8, 10,12,15,18,22,25,30,35,40],
                                                                                    [0.005,0.01, 0.03,0.05, 0.15, 0.3, 0.4,  0.6, 0.8],[0.005,0.01, 0.03,0.05, 0.15, 0.3, 0.4,  0.6, 0.8] ):
    now_ = (datetime.datetime.now() + datetime.timedelta(hours=9) ) .strftime('%m%d_%H%M')
    
    lda_main(now_, n_components=n_components, topic_word_prior=topic_word_prior, doc_topic_prior = doc_topic_prior, batch_size=batch_size, max_iter=25)
    gc.collect()
    
    

Fitting LDA models with tf features, n_samples=87733 and n_features=5050
iteration: 1 of max_iter: 25, perplexity: 1971.6696
iteration: 2 of max_iter: 25, perplexity: 1885.3969
iteration: 3 of max_iter: 25, perplexity: 1851.2197
iteration: 4 of max_iter: 25, perplexity: 1835.7455
iteration: 5 of max_iter: 25, perplexity: 1827.7021
iteration: 6 of max_iter: 25, perplexity: 1822.1569
iteration: 7 of max_iter: 25, perplexity: 1818.6281
iteration: 8 of max_iter: 25, perplexity: 1816.2183
iteration: 9 of max_iter: 25, perplexity: 1814.3506
iteration: 10 of max_iter: 25, perplexity: 1812.6123
iteration: 11 of max_iter: 25, perplexity: 1811.5839
iteration: 12 of max_iter: 25, perplexity: 1810.6091
iteration: 13 of max_iter: 25, perplexity: 1810.0052
iteration: 19 of max_iter: 25, perplexity: 1807.7450
iteration: 20 of max_iter: 25, perplexity: 1807.6765
done in 533.810s.
Fitting LDA models with tf features, n_samples=87733 and n_features=5050
iteration: 1 of max_iter: 25, perplexity: 1870.665

iteration: 15 of max_iter: 25, perplexity: 1459.8759
iteration: 16 of max_iter: 25, perplexity: 1457.8454
iteration: 17 of max_iter: 25, perplexity: 1456.0565
iteration: 18 of max_iter: 25, perplexity: 1454.4730
iteration: 19 of max_iter: 25, perplexity: 1453.0584
iteration: 20 of max_iter: 25, perplexity: 1451.7848
iteration: 21 of max_iter: 25, perplexity: 1450.6342
iteration: 22 of max_iter: 25, perplexity: 1449.5894
iteration: 23 of max_iter: 25, perplexity: 1448.6388
iteration: 24 of max_iter: 25, perplexity: 1447.7694
iteration: 25 of max_iter: 25, perplexity: 1446.9680
done in 715.862s.
Fitting LDA models with tf features, n_samples=87733 and n_features=5050
iteration: 1 of max_iter: 25, perplexity: 1748.0655
iteration: 2 of max_iter: 25, perplexity: 1634.8427
iteration: 3 of max_iter: 25, perplexity: 1574.0995
iteration: 4 of max_iter: 25, perplexity: 1537.4725
iteration: 5 of max_iter: 25, perplexity: 1514.0645
iteration: 6 of max_iter: 25, perplexity: 1498.1108
iteration: 7 o

iteration: 5 of max_iter: 25, perplexity: 1478.1384
iteration: 6 of max_iter: 25, perplexity: 1470.4374
iteration: 7 of max_iter: 25, perplexity: 1465.0643
iteration: 8 of max_iter: 25, perplexity: 1461.0530
iteration: 9 of max_iter: 25, perplexity: 1457.9959
iteration: 10 of max_iter: 25, perplexity: 1455.6584
iteration: 11 of max_iter: 25, perplexity: 1453.7738
iteration: 12 of max_iter: 25, perplexity: 1452.2245
iteration: 13 of max_iter: 25, perplexity: 1450.9139
iteration: 14 of max_iter: 25, perplexity: 1449.8215
iteration: 15 of max_iter: 25, perplexity: 1448.8316
iteration: 16 of max_iter: 25, perplexity: 1447.9975
iteration: 17 of max_iter: 25, perplexity: 1447.2605
iteration: 18 of max_iter: 25, perplexity: 1446.5822
iteration: 19 of max_iter: 25, perplexity: 1446.0171
iteration: 20 of max_iter: 25, perplexity: 1445.4890
iteration: 21 of max_iter: 25, perplexity: 1445.0220
iteration: 22 of max_iter: 25, perplexity: 1444.6147
iteration: 23 of max_iter: 25, perplexity: 1444.238

iteration: 9 of max_iter: 25, perplexity: 1535.7635
iteration: 10 of max_iter: 25, perplexity: 1534.9653
iteration: 11 of max_iter: 25, perplexity: 1534.3496
iteration: 12 of max_iter: 25, perplexity: 1533.8058
iteration: 13 of max_iter: 25, perplexity: 1533.5381
iteration: 14 of max_iter: 25, perplexity: 1533.3136
iteration: 15 of max_iter: 25, perplexity: 1533.0454
iteration: 16 of max_iter: 25, perplexity: 1532.8765
iteration: 17 of max_iter: 25, perplexity: 1532.7461
iteration: 18 of max_iter: 25, perplexity: 1532.6323
iteration: 19 of max_iter: 25, perplexity: 1532.5136
iteration: 20 of max_iter: 25, perplexity: 1532.4609
done in 531.318s.
Fitting LDA models with tf features, n_samples=87733 and n_features=5050
iteration: 1 of max_iter: 25, perplexity: 1618.0820
iteration: 2 of max_iter: 25, perplexity: 1536.1750
iteration: 3 of max_iter: 25, perplexity: 1509.9227
iteration: 4 of max_iter: 25, perplexity: 1497.9706
iteration: 5 of max_iter: 25, perplexity: 1491.7516
iteration: 6 o

iteration: 18 of max_iter: 25, perplexity: 1420.0054
iteration: 19 of max_iter: 25, perplexity: 1418.8207
iteration: 20 of max_iter: 25, perplexity: 1417.7502
Fitting LDA models with tf features, n_samples=87733 and n_features=5050
iteration: 1 of max_iter: 25, perplexity: 1828.5819
iteration: 2 of max_iter: 25, perplexity: 1774.1494
iteration: 3 of max_iter: 25, perplexity: 1758.0469
iteration: 4 of max_iter: 25, perplexity: 1752.3661
iteration: 5 of max_iter: 25, perplexity: 1749.3460
iteration: 6 of max_iter: 25, perplexity: 1747.2013
iteration: 7 of max_iter: 25, perplexity: 1746.1004
iteration: 8 of max_iter: 25, perplexity: 1745.3692
iteration: 9 of max_iter: 25, perplexity: 1744.5999
iteration: 10 of max_iter: 25, perplexity: 1744.6473
done in 285.203s.
Fitting LDA models with tf features, n_samples=87733 and n_features=5050
iteration: 1 of max_iter: 25, perplexity: 1736.1680
iteration: 2 of max_iter: 25, perplexity: 1680.8836
iteration: 3 of max_iter: 25, perplexity: 1664.3173


...

iteration: 24 of max_iter: 25, perplexity: 1487.6486
iteration: 25 of max_iter: 25, perplexity: 1487.2682
done in 567.449s.
Fitting LDA models with tf features, n_samples=87733 and n_features=5050
iteration: 1 of max_iter: 25, perplexity: 2361.5176
iteration: 2 of max_iter: 25, perplexity: 2196.5638
iteration: 3 of max_iter: 25, perplexity: 2132.3635
iteration: 4 of max_iter: 25, perplexity: 2106.9988
iteration: 5 of max_iter: 25, perplexity: 2097.2035
iteration: 6 of max_iter: 25, perplexity: 2093.8069
iteration: 7 of max_iter: 25, perplexity: 2092.0946
iteration: 8 of max_iter: 25, perplexity: 2092.2012
iteration: 9 of max_iter: 25, perplexity: 2091.8592
iteration: 10 of max_iter: 25, perplexity: 2091.7915
done in 296.891s.
Fitting LDA models with tf features, n_samples=87733 and n_features=5050
iteration: 1 of max_iter: 25, perplexity: 2174.7089
iteration: 2 of max_iter: 25, perplexity: 2009.8071
iteration: 3 of max_iter: 25, perplexity: 1943.1713
iteration: 4 of max_iter: 25, perpl

iteration: 11 of max_iter: 25, perplexity: 1446.9052
iteration: 12 of max_iter: 25, perplexity: 1442.2864
iteration: 13 of max_iter: 25, perplexity: 1438.4122
iteration: 14 of max_iter: 25, perplexity: 1435.1244
iteration: 15 of max_iter: 25, perplexity: 1432.3035
iteration: 16 of max_iter: 25, perplexity: 1429.8411
iteration: 17 of max_iter: 25, perplexity: 1427.6894
iteration: 18 of max_iter: 25, perplexity: 1425.7856
iteration: 19 of max_iter: 25, perplexity: 1424.0847
iteration: 20 of max_iter: 25, perplexity: 1422.5641
iteration: 21 of max_iter: 25, perplexity: 1421.1982
iteration: 22 of max_iter: 25, perplexity: 1419.9622
iteration: 23 of max_iter: 25, perplexity: 1418.8386
iteration: 24 of max_iter: 25, perplexity: 1417.8141
iteration: 25 of max_iter: 25, perplexity: 1416.8729
done in 686.559s.
Fitting LDA models with tf features, n_samples=87733 and n_features=5050
iteration: 1 of max_iter: 25, perplexity: 1915.7620
iteration: 2 of max_iter: 25, perplexity: 1696.6749
iteration:

iteration: 13 of max_iter: 25, perplexity: 1478.2097
iteration: 14 of max_iter: 25, perplexity: 1476.4626
iteration: 15 of max_iter: 25, perplexity: 1474.9859
iteration: 16 of max_iter: 25, perplexity: 1473.6914
iteration: 17 of max_iter: 25, perplexity: 1472.5331
iteration: 18 of max_iter: 25, perplexity: 1471.5120
iteration: 19 of max_iter: 25, perplexity: 1470.5957
iteration: 20 of max_iter: 25, perplexity: 1469.7747
iteration: 21 of max_iter: 25, perplexity: 1469.0259
iteration: 22 of max_iter: 25, perplexity: 1468.3694
iteration: 23 of max_iter: 25, perplexity: 1467.7505
iteration: 24 of max_iter: 25, perplexity: 1467.2141
iteration: 25 of max_iter: 25, perplexity: 1466.7122
done in 699.991s.
Fitting LDA models with tf features, n_samples=87733 and n_features=5050
iteration: 1 of max_iter: 25, perplexity: 1879.0793
iteration: 2 of max_iter: 25, perplexity: 1692.3374
iteration: 3 of max_iter: 25, perplexity: 1614.1216
iteration: 4 of max_iter: 25, perplexity: 1575.0056
iteration: 5

iteration: 10 of max_iter: 25, perplexity: 1827.4828
iteration: 11 of max_iter: 25, perplexity: 1827.3581
iteration: 12 of max_iter: 25, perplexity: 1827.2641
done in 350.881s.
Fitting LDA models with tf features, n_samples=87733 and n_features=5050
iteration: 1 of max_iter: 25, perplexity: 1802.9188
iteration: 2 of max_iter: 25, perplexity: 1696.4438
iteration: 3 of max_iter: 25, perplexity: 1659.5709
iteration: 4 of max_iter: 25, perplexity: 1641.3479
iteration: 5 of max_iter: 25, perplexity: 1630.8105
iteration: 6 of max_iter: 25, perplexity: 1624.0835
iteration: 7 of max_iter: 25, perplexity: 1619.8436
iteration: 8 of max_iter: 25, perplexity: 1617.2757
iteration: 9 of max_iter: 25, perplexity: 1615.6812
iteration: 10 of max_iter: 25, perplexity: 1614.2253
iteration: 11 of max_iter: 25, perplexity: 1613.1683
iteration: 12 of max_iter: 25, perplexity: 1612.4454
iteration: 13 of max_iter: 25, perplexity: 1611.9167
iteration: 14 of max_iter: 25, perplexity: 1611.2122
iteration: 15 of 

iteration: 25 of max_iter: 25, perplexity: 1364.9632
done in 572.891s.
Fitting LDA models with tf features, n_samples=87733 and n_features=5050
iteration: 1 of max_iter: 25, perplexity: 1861.0799
iteration: 2 of max_iter: 25, perplexity: 1640.0330
iteration: 3 of max_iter: 25, perplexity: 1541.3486
iteration: 4 of max_iter: 25, perplexity: 1488.7233
iteration: 5 of max_iter: 25, perplexity: 1457.4304
iteration: 6 of max_iter: 25, perplexity: 1437.0605
iteration: 7 of max_iter: 25, perplexity: 1422.7883
iteration: 8 of max_iter: 25, perplexity: 1412.2307
iteration: 9 of max_iter: 25, perplexity: 1404.1114
iteration: 10 of max_iter: 25, perplexity: 1397.6926
iteration: 11 of max_iter: 25, perplexity: 1392.5174
iteration: 12 of max_iter: 25, perplexity: 1388.2695
iteration: 13 of max_iter: 25, perplexity: 1384.7197
iteration: 14 of max_iter: 25, perplexity: 1381.7011
iteration: 15 of max_iter: 25, perplexity: 1379.0951
iteration: 16 of max_iter: 25, perplexity: 1376.8188
iteration: 17 of 

### results

### プロフィールを

In [ ]:
# def get_top_prof (topic_idx, top_n = 100):
    
#     topics = lda.transform(X)
#     prof_idx_list = topics[:, topic_idx].argsort()[:-top_n - 1:-1]
#     return [docs[d] for d in prof_idx_list]

In [ ]:
# get_top_prof(0, 20)

In [ ]:
# get_top_prof(1, 20)

In [ ]:
# get_top_prof(2, 20)

In [ ]:
# get_top_prof(3, 20)

In [ ]:
# get_top_prof(4, 20)

In [ ]:
# get_top_prof(5, 20)

## corpusとauthorをpickle

In [21]:
len(corpus)

87733

In [31]:
author_nonan = []
for i,doc in tqdm(enumerate(docs)):
    if str(doc) == "nan":
        continue
    else:
        author_nonan.append(df["id"].iloc[i])


print(author_nonan[:10])

93069it [00:01, 80988.49it/s]


[235517449,
 3004809181,
 960755580775276544,
 1326081427,
 1147981279,
 1018121710048964609,
 1165141243443068928,
 1171692111084699650,
 3265972872,
 849903469024518144]

In [41]:
dir_name  = "/home/input/twitter_api/experiment/0803expt/"

file_name = 'author_nonan.pickle'
with open(dir_name + file_name, mode="wb") as f:
    pickle.dump(author_nonan, f)
        

In [42]:
file_name = 'corpus.pickle'
with open(dir_name + file_name, mode="wb") as f:
    pickle.dump( corpus, f)